# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Olonkinbyen,70.9221,-8.7187,19.40,58,40,18.59,SJ,1732906145
1,1,Ushuaia,-54.8000,-68.3000,51.46,54,40,32.21,AR,1732906146
2,2,Fale old settlement,-9.3852,-171.2468,81.73,76,75,15.68,TK,1732906147
3,3,Grytviken,-54.2811,-36.5092,41.88,82,100,9.73,GS,1732906149
4,4,Saint Paul Harbor,57.7900,-152.4072,27.91,69,75,11.50,US,1732906150


In [3]:
# Date Cleaning w/dates
city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Olonkinbyen,70.9221,-8.7187,19.40,58,40,18.59,SJ,2024-11-29 18:49:05
1,1,Ushuaia,-54.8000,-68.3000,51.46,54,40,32.21,AR,2024-11-29 18:49:06
2,2,Fale old settlement,-9.3852,-171.2468,81.73,76,75,15.68,TK,2024-11-29 18:49:07
3,3,Grytviken,-54.2811,-36.5092,41.88,82,100,9.73,GS,2024-11-29 18:49:09
4,4,Saint Paul Harbor,57.7900,-152.4072,27.91,69,75,11.50,US,2024-11-29 18:49:10


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
city_data_df2 = city_data_df.dropna()

mask = (city_data_df2["Max Temp"] >= min_temp) & (city_data_df2["Max Temp"] < max_temp) & (city_data_df2["Wind Speed"] < max_wind)
city_data_df2 = city_data_df2.loc[mask].reset_index(drop=True)

# Display sample data
city_data_df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,16,Bento Gonçalves,-29.1714,-51.5192,77.52,76,99,1.01,BR,2024-11-29 18:49:24
1,18,Alice Springs,-23.7000,133.8833,74.75,50,100,5.75,AU,2024-11-29 18:49:26
2,21,Bu’aale,1.0833,42.5833,76.44,90,100,6.24,SO,2024-11-29 18:49:29
3,30,Douentza,14.9951,-2.9517,74.61,16,0,6.78,ML,2024-11-29 18:49:39
4,33,Tazacorte,28.6290,-17.9293,74.37,43,0,7.00,ES,2024-11-29 18:49:43
...,...,...,...,...,...,...,...,...,...,...
96,561,Boali,4.8005,18.1275,79.03,48,84,1.86,CF,2024-11-29 19:00:10
97,563,Soroti,1.7146,33.6111,70.88,64,67,3.80,UG,2024-11-29 19:00:12
98,574,Benguela,-12.5763,13.4055,77.81,78,90,2.10,AO,2024-11-29 19:00:25
99,576,Bandar Lampung,-5.4254,105.2580,76.39,90,100,1.95,ID,2024-11-29 19:00:27


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
hotel_rows = []

for index, row in city_data_df2.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [7]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,Bento Gonçalves,BR,-29.1714,-51.5192,"Hotel Laghetto Viverone Estação, Rua 10 de Nov...",Hotel Laghetto Viverone Estação,246.0,None,None
1,Alice Springs,AU,-23.7000,133.8833,"Aurora Alice Springs, 11 Leichhardt Terrace, A...",Aurora Alice Springs,48.0,None,www.auroraresorts.com.au
2,Bu’aale,SO,1.0833,42.5833,None,None,NaN,None,None
3,Douentza,ML,14.9951,-2.9517,None,None,NaN,None,None
4,Tazacorte,ES,28.6290,-17.9293,"App Leyma, Avenida Felipe Lorenzo, 8, 38770 Ta...",App Leyma,1250.0,None,None
...,...,...,...,...,...,...,...,...,...
96,Boali,CF,4.8005,18.1275,None,None,NaN,None,None
97,Soroti,UG,1.7146,33.6111,"Paradise Hotel, Old Mbale Road, Cell G, Uganda",Paradise Hotel,522.0,None,None
98,Benguela,AO,-12.5763,13.4055,"Hotel Moibela, Avenida 10 de Fevereiro, Bengue...",Hotel Moibela,147.0,None,None
99,Bandar Lampung,ID,-5.4254,105.2580,"Grand Anugerah, Jalan Raden Intan, Bandar Lamp...",Grand Anugerah,495.0,None,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
target_city = "Clearwater, Florida, United States of America"
params = {
    "text": target_city,
    "format": "json",
    "apiKey": geoapify_key
}
#Set base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [9]:
# Run request
response = requests.get(base_url, params=params)
print(response.status_code)

# Print the json (pretty printed)
data = response.json()
pprint(data)

200
{'query': {'parsed': {'city': 'clearwater',
                      'country': 'united states of america',
                      'expected_type': 'city',
                      'state': 'florida'},
           'text': 'Clearwater, Florida, United States of America'},
 'results': [{'address_line1': 'Clearwater (Clearwater), FL',
              'address_line2': 'United States of America',
              'bbox': {'lat1': 27.9351239,
                       'lat2': 28.0499611,
                       'lon1': -82.8434471,
                       'lon2': -82.6792432},
              'category': 'administrative',
              'city': 'Clearwater',
              'country': 'United States',
              'country_code': 'us',
              'county': 'Pinellas County',
              'datasource': {'attribution': '© OpenStreetMap contributors',
                             'license': 'Open Database License',
                             'sourcename': 'openstreetmap',
                             'url'

In [10]:
# Extract lat/lon
results = data.get("results", [])

if len(results) > 0:
    location = results[0]
    latitude = location.get("lat")
    longitude = location.get("lon")
    address = location.get("formatted")

    # Print results
    print(f"{address} is located at {latitude}, {longitude}")
else:
    print("OH NOOOO!! YOUR RESPONSE DID NOT INCLUDE ANY RESULTS:'(")

Clearwater (Clearwater), FL, United States of America is located at 27.9658533, -82.8001026


In [12]:
# Set parameters to search for a hotel
target_city = "Clearwater, Florida, United States of America"
params = {
    "text": target_city,
    "format": "json",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = 27.9658533
    longitude = -82.8001026

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 20

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    print(response.status_code)

    # Convert the API response to JSON format
    name_address = response.json()
    pprint(data)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
200
{'query': {'parsed': {'city': 'clearwater',
                      'country': 'united states of america',
                      'expected_type': 'city',
                      'state': 'florida'},
           'text': 'Clearwater, Florida, United States of America'},
 'results': [{'address_line1': 'Clearwater (Clearwater), FL',
              'address_line2': 'United States of America',
              'bbox': {'lat1': 27.9351239,
                       'lat2': 28.0499611,
                       'lon1': -82.8434471,
                       'lon2': -82.6792432},
              'category': 'administrative',
              'city': 'Clearwater',
              'country': 'United States',
              'country_code': 'us',
              'county': 'Pinellas County',
              'datasource': {'attribution': '© OpenStreetMap contributors',
                             'license': 'Open Database License',
                             'sourcename': 'openstreetmap',
            

KeyError: 'City'

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
# Convert the data to a DataFrame
hotel_df = pd.DataFrame(data)

# Create a Plotly map
fig = px.scatter_geo(df, lat='lat', lon='lon', text='city',
                     hover_data={'city': False, 'hotel_name': True, 'country': True},
                     title="Cities with Hotels")

# Show the map
fig.show()

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.